![texto alternativo](https://raw.githubusercontent.com/Chilangdon20/PYTHON/master/Curso%20Basico/CursoExpress/Imagenes/PANDAS.jpg)

# Optimización Basica en Pandas.

## Introdución:

Hasta ahora nos hemos centrado en las estructuras de datos integradas en Python,ahora cambiaremos de marcha y nos centraremos en una de las heramientas de análisis de datos más populares: El PANDAS DATAFRAME.

Cabe mencionar que es necesario que estes familizarizado con Pandas antes de continuar, de lo contrario te recomiendo estudiar un poco acerca de esta libreria.

Recordemos que pandas es una biblioteca utilizada para el análisis de datos, la construcción principal de pandas es el *DataFrame* , una estructura de datos tabular y con filas y columnas etiquetadas.

En esta lectura nos centraremos en los mejores enfoques para iterar sobre un DataFrame.
 
Comencemos analizando un conjunto de datos de Major League Baseball,el dataset contiene estadisticas del equipo para cada equipo de Major League Baseball del año 1962 a 2012.

La columna de equipo es el nombre abreviado de cada equipo de béisbol, el mprimer equipo representa a los Dimondbacks de Arizona(ARI),todas las columnas de este dataframe representan estadísticas especifícas para cada equipo en un año o temporada determinados.

Por ahora nos centraremos en la columna W , que especifica la cantdad de victorias que tuvo un equipo y la columna G que contiene la cantidad de juegos que un equipo jugo en una temporada.

In [2]:
import pandas as pd
data = pd.read_csv("https://assets.datacamp.com/production/repositories/3581/datasets/779033fb8fb5021aee9ff46253980abcbc5851f3/baseball_stats.csv")

In [ ]:
print(data.head())

  Team League  Year   RS   RA  ...  RankSeason  RankPlayoffs    G   OOBP   OSLG
0  ARI     NL  2012  734  688  ...         NaN           NaN  162  0.317  0.415
1  ATL     NL  2012  700  600  ...         4.0           5.0  162  0.306  0.378
2  BAL     AL  2012  712  705  ...         5.0           4.0  162  0.315  0.403
3  BOS     AL  2012  734  806  ...         NaN           NaN  162  0.331  0.428
4  CHC     NL  2012  613  759  ...         NaN           NaN  162  0.335  0.424

[5 rows x 15 columns]


In [ ]:
# Observamos todos los nombres de las columnas
print(data.columns.values)

['Team' 'League' 'Year' 'RS' 'RA' 'W' 'OBP' 'SLG' 'BA' 'Playoffs'
 'RankSeason' 'RankPlayoffs' 'G' 'OOBP' 'OSLG']


Una estadistica popular utilizada para evaluar el rendimiento de un equipo para una temporada determinada es el porcentaje de victorias del equipo, esta métrica se calcula dividiendo las victorias totales de un equipo por el número de juegos jugados,para esto haremos una simple función para realizar este cálculo.

In [4]:
import numpy as np

def calc_vict_porc(vict,juegos_jugados):

  vict_porcentaje = vict / juegos_jugados

  return np.round(vict_porcentaje,2)

Si un equipo gana 50 de cada 100 juegos , vemos que nuestra función retorna el resultado correcto.

In [ ]:
vict_porce = calc_vict_porc(50,100)
print(vict_porce)

0.5


Nos gustaria crear una nueva columna a nuestro DataFrame ``data`` que almacene el porcentaje de victorias de cada equipo durante una temporada, para hacer esto necesitaremos iterar sobre las filas del dataframe y aplicar nuestra función creada anteriormente.

Primero creamos una lista vacia para almacenar todos los porcentajes de victorias que calcularemos , despues escribiremos un loop que iterará sobre cada fia del DataFrame.


**IMPORTANTE** veanos que estamos usando una variable de indice *i* que varia desde cero hasta el número de filas que existen dentro del DataFrame.

Luego usaoms el método ``.iloc`` para buscar cada fila individual dentro del dataframe usando la variable del índice.

Ahora tomaremos las victorias y los juegos de cada equipo haciendo referencia a las columnas ``W`` & ``G``.Despues pasamos las victorias y los juegos del equipo a nuestra lista vacia para calculas los porcentajes de victorias.

Finalmente ,agregamos a nuestra lista ``.append()``y continuamos el ciclo.

Creamos nuestra columna deseada  en el DataFrame llamada ``WP``, estableciendo el valor de la columna igual a ``vict_poc_list``

In [ ]:
vict_poc_list = []

for i in range(len(data)):

  row = data.iloc[i]

  vict = row['W']
  juegos_jugados = row['G']

  vict_porc = calc_vict_porc(vict,juegos_jugados)

  vict_poc_list.append(vict_porc)

data['WP'] = vict_poc_list

In [ ]:
print(data.head())

  Team League  Year   RS   RA  ...  RankPlayoffs    G   OOBP   OSLG    WP
0  ARI     NL  2012  734  688  ...           NaN  162  0.317  0.415  0.50
1  ATL     NL  2012  700  600  ...           5.0  162  0.306  0.378  0.58
2  BAL     AL  2012  712  705  ...           4.0  162  0.315  0.403  0.57
3  BOS     AL  2012  734  806  ...           NaN  162  0.331  0.428  0.43
4  CHC     NL  2012  613  759  ...           NaN  162  0.335  0.424  0.38

[5 rows x 16 columns]


Al imprimir las cinco filas de nuestro DataFrame , vemos que ahora aparece la columna de porcentaje de victorias.

Recorres el dataframe con ``.iloc`` nos dio el resultado deseado pero... ¿ Es eficiente?


In [ ]:
%%timeit
vict_poc_list = []

for i in range(len(data)):

  row = data.iloc[i]

  vict = row['W']
  juegos_jugados = row['G']

  vict_porc = calc_vict_porc(vict,juegos_jugados)

  vict_poc_list.append(vict_porc)

data['WP'] = vict_poc_list

1 loop, best of 3: 273 ms per loop


Al estimar el tiempo de ejecución , ``.iloc`` tomo 273 milisegundos , lo cual es bastante ineficiente.Pero no todo esta perdido ya que panda viene con algunos métodoe eficientes para recorrer un DataFrame.

El primer método que analizaremos sera el método ``.iterrows()``.

## ``.iterrows()``

Este método es similar al metodo ``.iloc()``, pero ``.iterrows()`` devuelve cada fila del DatafFrame como una tupla de pares, esto significa que cada objeto devuelto por ``.iterrows()``contiene el indice de cada fila como el mprimer elemento y los datos en cada fila como una serie de pandas como el segundo elemento.

In [ ]:
vict_porcent_lista = []
for i,row in data.iterrows():

Veamos que ya creamos la lista vacia pero ahora tenemos que crear una variable de indice para buscar cada fila dentro del DataFrame, afortunadamente ``.iterrows()``maneja la indexación por nosotros, el resto del ciclo permanece exatcamente igual que el anterior.

In [ ]:
vict_porcent_lista = []

for i,row in data.iterrows():

   vict = row['W']
   juegos_jugados = row['G']

   vict_porc = calc_vict_porc(vict, juegos_jugados)

   vict_porcent_lista.append(vict_porc)

data['WP'] = vict_porcent_lista

In [ ]:
print(vict_porcent_lista)

[0.5, 0.58, 0.57, 0.43, 0.38, 0.52, 0.6, 0.42, 0.4, 0.54, 0.34, 0.44, 0.55, 0.53, 0.43, 0.51, 0.41, 0.46, 0.59, 0.58, 0.5, 0.49, 0.47, 0.46, 0.58, 0.54, 0.56, 0.57, 0.45, 0.6, 0.58, 0.55, 0.43, 0.56, 0.44, 0.49, 0.49, 0.49, 0.45, 0.59, 0.44, 0.35, 0.44, 0.53, 0.51, 0.59, 0.39, 0.48, 0.6, 0.46, 0.63, 0.44, 0.44, 0.41, 0.53, 0.56, 0.56, 0.59, 0.5, 0.5, 0.4, 0.56, 0.41, 0.55, 0.46, 0.54, 0.56, 0.43, 0.51, 0.5, 0.49, 0.47, 0.41, 0.49, 0.49, 0.48, 0.58, 0.49, 0.59, 0.5, 0.6, 0.35, 0.56, 0.38, 0.57, 0.53, 0.59, 0.56, 0.52, 0.43, 0.43, 0.53, 0.4, 0.59, 0.52, 0.49, 0.48, 0.4, 0.57, 0.53, 0.54, 0.46, 0.4, 0.6, 0.59, 0.49, 0.53, 0.43, 0.64, 0.46, 0.57, 0.39, 0.46, 0.52, 0.54, 0.56, 0.52, 0.54, 0.46, 0.36, 0.51, 0.44, 0.42, 0.59, 0.6, 0.55, 0.46, 0.5, 0.46, 0.46, 0.52, 0.53, 0.46, 0.62, 0.52, 0.56, 0.54, 0.55, 0.55, 0.47, 0.57, 0.41, 0.39, 0.38, 0.44, 0.53, 0.6, 0.49, 0.53, 0.37, 0.56, 0.52, 0.43, 0.59, 0.52, 0.44, 0.44, 0.59, 0.55, 0.54, 0.44, 0.45, 0.43, 0.58, 0.51, 0.51, 0.49, 0.54, 0.58, 0.47

Podemos observar en la siguiente linea de código que el uso de ``.iterrows()``lleva aproximadamente la mitas de tiempo que ``.iloc()`` para iterar sobre nuestros datos.

In [ ]:
%%timeit
vict_porcent_lista = []

for i,row in data.iterrows():

   vict = row['W']
   juegos_jugados = row['G']

   vict_porc = calc_vict_porc(vict, juegos_jugados)

   vict_porcent_lista.append(vict_porc)

data['WP'] = vict_porcent_lista

10 loops, best of 3: 154 ms per loop




---



---



# Metodo ``itertuples()``

Anteriormente , cubrimos cómo iterar sobre un DataFrame de pandas fila por fila usando el metodo ``.iterrows()`` , pandas también viene con un método de iteraciónn similas llamado ``.itertuples()``.

Supongamos que tenemos un DataFrame de pandas llamado team_wins_df que contiene las victoria totales de cada equipo en una temporada.

In [ ]:
#Seleccionamos las columnas que necesitamos
team_winds_df = data.loc[:,['Team','Year','W']]

print(team_winds_df.head())

  Team  Year   W
0  ARI  2012  81
1  ATL  2012  94
2  BAL  2012  93
3  BOS  2012  69
4  CHC  2012  61


Si usamos ``.iterrows()`` para recorrer nuestro DataFrame e imprimir en la tupla de cada fila , vemos que los valores de cada fila se almcena como una serie de pandas, recordemos que ``.iterrows()`` devuelve cada fila del DataFrame como una tupla de (index,pandas), por lo que tenemos que acceder a los valores de la fila con indexación de corchetes.

In [ ]:
for r_tupla in team_winds_df.iterrows():

  print(r_tupla)
  print(type(r_tupla[1]))

Pero, en su lugar podriamos usar ``.itertuples()``para recorrer nuestras filas del DF.El metodo ``.itertuples()``devuelve cada fila del DataFrame como un tipo de datos especia llamado *namedtuple*-(TuplaNombrada/TN)


Una TN es uno de los tipos de datos especializados que existen dentro del módulo de colecciones que hemos discutido anteriormente.Estos tipos de datos se comportan como una tupla de Python pero tienen campos accesibles mediante la búsqueda de atributos.

¿Que significa esto?
 Veamos en el resultado que cada fila impresa tiene un indice de atributo y cada columna  tambien,estos ignifica que podemos acceder acada uno de estos atributos con una búsqueda utilizando el metodo de puntos.

In [ ]:
for r_namedtuple in team_winds_df.itertuples():
  print(r_namedtuple)

Pandas(Index=0, Team='ARI', Year=2012, W=81)
Pandas(Index=1, Team='ATL', Year=2012, W=94)
Pandas(Index=2, Team='BAL', Year=2012, W=93)
Pandas(Index=3, Team='BOS', Year=2012, W=69)
Pandas(Index=4, Team='CHC', Year=2012, W=61)
Pandas(Index=5, Team='CHW', Year=2012, W=85)
Pandas(Index=6, Team='CIN', Year=2012, W=97)
Pandas(Index=7, Team='CLE', Year=2012, W=68)
Pandas(Index=8, Team='COL', Year=2012, W=64)
Pandas(Index=9, Team='DET', Year=2012, W=88)
Pandas(Index=10, Team='HOU', Year=2012, W=55)
Pandas(Index=11, Team='KCR', Year=2012, W=72)
Pandas(Index=12, Team='LAA', Year=2012, W=89)
Pandas(Index=13, Team='LAD', Year=2012, W=86)
Pandas(Index=14, Team='MIA', Year=2012, W=69)
Pandas(Index=15, Team='MIL', Year=2012, W=83)
Pandas(Index=16, Team='MIN', Year=2012, W=66)
Pandas(Index=17, Team='NYM', Year=2012, W=74)
Pandas(Index=18, Team='NYY', Year=2012, W=95)
Pandas(Index=19, Team='OAK', Year=2012, W=94)
Pandas(Index=20, Team='PHI', Year=2012, W=81)
Pandas(Index=21, Team='PIT', Year=2012, W=79

Aqui,podemos imprimir el último indice usando ``.index``


In [ ]:
print(r_namedtuple.Index)

1231


Tambien podemos imprimir  el equipo con ``.Team``


In [ ]:
print(r_namedtuple.Team)

WSA


Cuando comprarmos ``.iterrows()`` con `ìtertuples()``vemos que hay basntante mejora

In [ ]:
%%timeit

for r_tupla in team_winds_df.iterrows():
    print(r_tupla)
  

In [ ]:
%%timeit
for r_namedtuple in team_winds_df.itertuples():
  print(r_namedtuple)

La razon por la que ``itertuples`` es mas eficiente se debe a la forma en que cada método almacena su salida dado que ``iterrows`` devuelve mas valores de cada fila como una serie de pandas hay mas sobrecarga.

Una nota rápida más sobre las diferencas entre estos métodos,cuando usamos ``iterrows`` podemos usar corcehetes para hacer referenc ia a una columna dentro de nuestro datafrane.

In [ ]:
for r_tupla in team_winds_df.iterrows():
    print(r_tupla[1]['Team'])

Si usamos la misma sintaxis con ``.itertuples``obtenemos un error .

In [ ]:
for r_namedtuple in team_winds_df.itertuples():
  print(r_namedtuple['Team'])

TypeError: ignored

Pero lo podriamos solucionar de la siguiente manera:

In [ ]:
for r_namedtuple in team_winds_df.itertuples():
  print(r_namedtuple.Team)



---



---

# Pandas que alteran el Looping.

Para poder eacribir código eficiente, queremos evitar los bucles cuando sea posible.Exploraremos una alterantiva al uso de ``.iterrows()``& ``.itertuples``para realizar cálculos en un DataFrame.

Conitnuaremos usando nuestro DatSet de béisbol y volveremos a visitar la función ``calc_vict_porc`` que hemos usado en el pasado.Nos gustaria crear una nueva ciolumna en nuestro DF llamado *RD* que almacena las diferencias de ejecución decada equipo a lo largo de dos años.

En lecciones anteriores , hicimos esto con un bucle for utilizando ``.iterrows`` o ``.itertuples``, aqui usarmeos ``.iterrows``como ejemplo.

Observemos que estamos iterando sobre data con un bucle for , pasando las columnas ``RS`` &  ``RA``de cada fila en nuestra función y luego agregando el resultado a cad fila de nuestra lista.




In [7]:
dif_iterrows = []
for i,col in data.iterrows():
  dife = calc_vict_porc(col['RS'], col['RA'])
  dif_iterrows.append(dife)

data['RD'] = dif_iterrows
print(data)

     Team League  Year   RS   RA  ...  RankPlayoffs    G   OOBP   OSLG    RD
0     ARI     NL  2012  734  688  ...           NaN  162  0.317  0.415  1.07
1     ATL     NL  2012  700  600  ...           5.0  162  0.306  0.378  1.17
2     BAL     AL  2012  712  705  ...           4.0  162  0.315  0.403  1.01
3     BOS     AL  2012  734  806  ...           NaN  162  0.331  0.428  0.91
4     CHC     NL  2012  613  759  ...           NaN  162  0.335  0.424  0.81
...   ...    ...   ...  ...  ...  ...           ...  ...    ...    ...   ...
1227  PHI     NL  1962  705  759  ...           NaN  161    NaN    NaN  0.93
1228  PIT     NL  1962  706  626  ...           NaN  161    NaN    NaN  1.13
1229  SFG     NL  1962  878  690  ...           2.0  165    NaN    NaN  1.27
1230  STL     NL  1962  774  664  ...           NaN  163    NaN    NaN  1.17
1231  WSA     AL  1962  599  716  ...           NaN  162    NaN    NaN  0.84

[1232 rows x 16 columns]


Esto nos da el restulado , pero no es la manera más eficiente,una alternativa al uso de un bucle para iterar sobre un DataFrame es usar el método de pandas ``.apply()``.

Esta función actúa como la funcion *map* que hemos usado en el pasado,toma esa función como entrada y la aplica al DataFrame completo.Dado que trabajamos con dato tabulares debemos especificar un eje en el que nos gustaria que actúe nuestra función,usar un ``0``para columnas o un ``1``para aplicarlo en las filas.

Al igual que la funcion ``map``, el metodo ``apply`` se puede usar con funciones anonimas o lambda.A continuación veamos cómo usariamos el méodo de aplicación de puntos para calcular nuestro diferenciales de carrera.

* Primero llamamod ``.apply``en nuestro conjunto de datos.

* Luego,usamos una funcion anonima , para iterar sobre las filas del DataFrame.
  **Notemos que nuestro argumento para lambda es row, ya que estamos aplicando a cada fila del dataFrame**

  Para cada fila, tomamos las columnas ``RS``y ``RA``y las pasamos a nuestra función ``calc_vict_porc``-

* Por ultimo especificamos nuestro eje para indicar que queremos iterar sobre filas en lugar de columnas.



In [8]:
data.apply(
    
    lambda row: calc_vict_porc(row['RS'], row['RA']),

    axis = 1
)

0       1.07
1       1.17
2       1.01
3       0.91
4       0.81
        ... 
1227    0.93
1228    1.13
1229    1.27
1230    1.17
1231    0.84
Length: 1232, dtype: float64

Cuando usamos el método de aplicación de puntos para calcilas las diferencia de ejecucion no necesitamos utilizar un bucle for ,podemos recopilas nuestros datos de ejecucion directamente en un objeto llamado ``dif_apply``


In [11]:
dif_apply = data.apply(
    lambda row: calc_vict_porc(row['RS'],row['RA']),
                               axis=1
)

data['RD'] = dif_apply
print(data)

     Team League  Year   RS   RA  ...  RankPlayoffs    G   OOBP   OSLG    RD
0     ARI     NL  2012  734  688  ...           NaN  162  0.317  0.415  1.07
1     ATL     NL  2012  700  600  ...           5.0  162  0.306  0.378  1.17
2     BAL     AL  2012  712  705  ...           4.0  162  0.315  0.403  1.01
3     BOS     AL  2012  734  806  ...           NaN  162  0.331  0.428  0.91
4     CHC     NL  2012  613  759  ...           NaN  162  0.335  0.424  0.81
...   ...    ...   ...  ...  ...  ...           ...  ...    ...    ...   ...
1227  PHI     NL  1962  705  759  ...           NaN  161    NaN    NaN  0.93
1228  PIT     NL  1962  706  626  ...           NaN  161    NaN    NaN  1.13
1229  SFG     NL  1962  878  690  ...           2.0  165    NaN    NaN  1.27
1230  STL     NL  1962  774  664  ...           NaN  163    NaN    NaN  1.17
1231  WSA     AL  1962  599  716  ...           NaN  162    NaN    NaN  0.84

[1232 rows x 16 columns]


Despues de crear nuestra nueva columna e imprimir el dataframe nosotros,veremos que nuestros resultados son idénticos a los enfoques anteriores.

# Iterarndo Óptimamente en Pandas.

Hemos recorrido un largo camino desde nuestro primer enfoque ``.iloc``para iterar sobre un DataFrame,cada enfoque que hemos discutido realmente ha mejorado el rendimiento.Pero estos enfoques se centrarn enrealizar cálculos para cada fila de nuestro DataFrame individualmente.

En esta sección exploraremos algunos elementos internos de panas que nos permiten realizar cálculos aún más eficientemente.

Como sabemos , debemos tratar de mantenernos alejados de los bucles cuando escribimos código en Python y trabajar con Pandas no es la excepción.

En las secciones anteriores , estábamos iterando sobre un DataFrame fila por fila para realizar un cálculo.Pandas es una biblioteca que se basa en NumPy,esto significa que cada Pnadas DataFrame que usamos , puede aprovechar las caracteristicas eficientes de las matrices de Numpy.

## ¿ Recuerdas la funcionalidad de transmisión de una matriz?

La transmisión permite que las matrices NumPy vectoricen las operaciones, por lo que se realizan en todos los ejmentos de un objeto a la vez.

Esto nos permite realizar cálculos de manera eficiente en matrices enteras,exploremos estos conceptos con algunos ejemplos.



In [16]:
print(data)

     Team League  Year   RS   RA  ...  RankPlayoffs    G   OOBP   OSLG    RD
0     ARI     NL  2012  734  688  ...           NaN  162  0.317  0.415  1.07
1     ATL     NL  2012  700  600  ...           5.0  162  0.306  0.378  1.17
2     BAL     AL  2012  712  705  ...           4.0  162  0.315  0.403  1.01
3     BOS     AL  2012  734  806  ...           NaN  162  0.331  0.428  0.91
4     CHC     NL  2012  613  759  ...           NaN  162  0.335  0.424  0.81
...   ...    ...   ...  ...  ...  ...           ...  ...    ...    ...   ...
1227  PHI     NL  1962  705  759  ...           NaN  161    NaN    NaN  0.93
1228  PIT     NL  1962  706  626  ...           NaN  161    NaN    NaN  1.13
1229  SFG     NL  1962  878  690  ...           2.0  165    NaN    NaN  1.27
1230  STL     NL  1962  774  664  ...           NaN  163    NaN    NaN  1.17
1231  WSA     AL  1962  599  716  ...           NaN  162    NaN    NaN  0.84

[1232 rows x 16 columns]


Como pandas esta contruido sobre NumPy , podemos tomar cualquiera de estos valores del DataFrame como una matriz de NumPy utilizando el método ``.values``

Aqui estamos recopilano los valores de la columna W en una matriz Numpy llamada vict_np,cuando imprimimos el tipo de dato vemos que es una matriz NumPy

In [14]:
vict_np = data['W'].values
print(type(vict_np))

<class 'numpy.ndarray'>


La belleza de saber que Pandas se bada en Numpy puede ser visto al aprovechar las habilidades de transmisión de una matriz NumPy,recordemos que podemos vectorizar nuestros calculos y realizarlos en conjuntos completos de una vez.

En lugar de recorer un DataFrame y tratar cada fila de forma independiente, colo lo hemos hecho con ``.iterrows``.

Podemos realizar cálculos en lsa matrices NumPy subyacentes a nuestro conjunto de datos ``data``.



In [15]:
data['RS'].values - data['RA'].values

array([  46,  100,    7, ...,  188,  110, -117])

Podemos ver que nuestro código se vuielve mucho más legible,aqui podemos ver como se calculan las diferencias de ejecucción.

In [17]:
dife_np = data['RS'].values - data['RA'].values
data['RD'] = dife_np
print(data)

     Team League  Year   RS   RA  ...  RankPlayoffs    G   OOBP   OSLG   RD
0     ARI     NL  2012  734  688  ...           NaN  162  0.317  0.415   46
1     ATL     NL  2012  700  600  ...           5.0  162  0.306  0.378  100
2     BAL     AL  2012  712  705  ...           4.0  162  0.315  0.403    7
3     BOS     AL  2012  734  806  ...           NaN  162  0.331  0.428  -72
4     CHC     NL  2012  613  759  ...           NaN  162  0.335  0.424 -146
...   ...    ...   ...  ...  ...  ...           ...  ...    ...    ...  ...
1227  PHI     NL  1962  705  759  ...           NaN  161    NaN    NaN  -54
1228  PIT     NL  1962  706  626  ...           NaN  161    NaN    NaN   80
1229  SFG     NL  1962  878  690  ...           2.0  165    NaN    NaN  188
1230  STL     NL  1962  774  664  ...           NaN  163    NaN    NaN  110
1231  WSA     AL  1962  599  716  ...           NaN  162    NaN    NaN -117

[1232 rows x 16 columns]


Podemos ver que neustros resultados son ídenticos a todos los otros enfoques.